In [1]:
import sys
sys.path.append('../src')


# OPTIONAL: Load the "autoreload" extension so that code can change
%reload_ext autoreload

# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

import cv2
import copy
import matplotlib.pyplot as plt
from matplotlib import patches
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view as swv
import os
from pathlib import Path
from skimage.transform import resize, downscale_local_mean
from sys import modules
import tifffile as tiff
from time import time
from tqdm import tqdm


from utils import list_subdir_filter as lsd, get_id
from metadata import metadata

md = metadata()

In [2]:

def get_circular_kernel(diameter):
    """
    Yeah also this. I honestly have no clue how it works but I guess I did back then.
    """
    mid = (diameter - 1) / 2
    distances = np.indices((diameter, diameter)) - np.array([mid, mid])[:, None, None]
    kernel = ((np.linalg.norm(distances, axis=0) - mid) <= 0).astype(int)
   
    return kernel


def fill_holes(img):
    """
    At least the names are easy to understand even for dumb today me. Thanks smarter yesterday me
    """
    des = copy.copy(img)
    contour, hier = cv2.findContours(des, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contour:
        cv2.drawContours(des, [cnt], 0, 255, -1)

    return des


# def n_components(bin_img):
#     comps = cv2.connectedComponents(bin_img, connectivity=8)[0]
#     holes = cv2.connectedComponents(~bin_img, connectivity=8)[0]
#     _min=1
#     _max=np.shape(bin_img).max//2
#     while comps>1:
#         _mid = (_min + _max) // 2
    

def remove_small_components(img, min_size=150, export_components=False):
    """
    Please by the love of god be a function that removes small components.
    """
    
    nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(img, connectivity=8)
    # connectedComponentswithStats yields every seperated component with information on each of them,
    # such as size. the following part is just taking out the background which is also considered 
    # a component, but most of the time we don't want that.
    rdict = {'n':nb_components, 'o':output, 's':stats, 'c':centroids}
    sizes = stats[1:, -1]; nb_components = nb_components - 1
    
    img2 = np.zeros((output.shape))
    # for every component in the image, you keep it only if it's above min_size. I guess the +1 is because
    # we removed the background? YES that's why the +1
    for i in range(0, nb_components):
        if sizes[i] >= min_size:
            img2[output == i + 1] = 255
    return img2, rdict


In [3]:
aaa ='lel'
print(f'{aaa:>5};')

  lel;


In [26]:
def unique_file(code, folder):
    files = lsd(folder, True, f'{code}.*tif[f]?')
    if len(files) == 1:
        return files[0]
    raise ValueError(f'Non-univocal file for {code} in {folder} - {len(files)}')

def odd(n):
    if n % 2 == 0:
        n += 1
    return int(n)

def print_duration(t, text):
    print(f'{text:.>12} --- {round(time() - t, 1)}s')
    return time()

def create_mask(
    img, 
    norm_percentile=80, 
    pool_size=16, 
    size_close_k=30, 
    size_open_k=12, 
    min_component_size= 2 * 10 ** 6, 
    debug_factor=0
):
    """
    basically, everything happens here. I'll write something better later.
    """

    blur_factor1 = odd(111 // (pool_size / 16))
    blur_factor2 = odd(151 // (pool_size / 16))
    
    orig_size = np.shape(img)
    print(orig_size)
    if img.dtype == np.uint16:
        img = (img >> 8).astype('uint8')
    norm_percentile = min(norm_percentile, 99.5)
    close_kernel = get_circular_kernel(size_close_k).astype('uint8')
    open_kernel  = get_circular_kernel(size_open_k).astype('uint8')
    
    # some pooling to reduce the size of the image. would functions other than 'average' be better? idk
    t = time()
    im_pool = downscale_local_mean(img, pool_size, pool_size)
    t = print_duration(t, 'pool')
    print(np.shape(im_pool))
    
    im_blur = cv2.GaussianBlur(im_pool, (blur_factor1, blur_factor1), 0)
    t = print_duration(t, 'blur')

    chosen_quantile = np.percentile(im_blur[im_blur>0], norm_percentile)
    im_norm = ((np.clip(im_blur, 0, chosen_quantile) / chosen_quantile) * 255).astype('uint8')
    t = print_duration(t, 'norm')

    im_bin = (cv2.threshold(im_norm, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]).astype('uint8')
    t = print_duration(t, 'bin')

    im_blur2 = cv2.GaussianBlur(im_bin, (blur_factor2, blur_factor2), 0).astype('uint8')
    t = print_duration(t, 'blur2')

    im_fill = fill_holes(im_blur2) 
    t = print_duration(t, 'fill')

    im_upscale = cv2.resize(src=im_fill, dsize=orig_size[::-1])
    t = print_duration(t, 'upscale')

    im_clean, components_dict = remove_small_components(im_upscale, min_component_size)
    t = print_duration(t, 'clean')

    if debug_factor:
        return {
            'pool' :im_pool[::debug_factor, ::debug_factor],
            'blur' :im_blur[::debug_factor, ::debug_factor],
            'norm' :im_norm[::debug_factor, ::debug_factor],
            'bin'  :im_bin[::debug_factor, ::debug_factor],
            'blur2':im_blur2[::debug_factor, ::debug_factor],
            'fill' :im_fill[::debug_factor, ::debug_factor],
            # 'up'   :im_upscale[::debug_factor, ::debug_factor],
            'clean':im_clean[::debug_factor, ::debug_factor],
            'fil'  :im_clean,
            'components':components_dict,
            'quantile':chosen_quantile,
        }

    return {'fil':im_clean, 'components':components_dict}



### from 01-remove bg

def how_old(path):
    age = int((time() - os.path.getmtime(path)) // 3600)
    if age < 1:
        return '<1h'
    if age < 24:
        return f'{age}h'
    age = int(age // 24)
    if age < 365:
        return f'{age}d'
    return '>1y'



def subtract_AF(signal, af, img_id, marker_name, clipping_threshold=20000):
    """
    uhu
    """

    def normalize_clipped(array, clipping_threshold):
        return np.clip(array, 0, clipping_threshold)/clipping_threshold

    
    def combine_clean_AF_channels(signal_minus_AF, AF, clipping_threshold, scale_factor=8):
        """
        I think this is for visualization purposes? so we get a jpg?
        """
        combined_RGB = [
            normalize_clipped(signal_minus_AF[::scale_factor, ::scale_factor], clipping_threshold),
            normalize_clipped(AF[::scale_factor, ::scale_factor], clipping_threshold),
            normalize_clipped(signal_minus_AF[::scale_factor, ::scale_factor], clipping_threshold)
        ]
    
        return combined_RGB

    
    def export_img_signal_histogram(signal_minus_AF, sample_id, marker_name):
        """
        given an image/array, well, export its intensity histogram as a png
        """
        plt.figure(figsize=(10,4))
        p = plt.hist(signal_minus_AF.flatten()[::4], log=True, bins=150, color='green')
        plt.title(f'{sample_id} ({marker_name}) post-AF correction')
        plt.vlines(x=0, ymin=0.1, ymax=p[0].max()*1.02, color='gold')
        plt.xlim(-65535, 65535)
        plt.savefig(os.path.join(md.folders['bg_removed'], f'intensity_dist_AF_correction_{sample_id}_{marker_name}.png'))
        plt.close()

    
    def export_img_signal_AF_overlap(rgb_array, sample_id, marker_name):
        """
        very simple tool to save the png of the overlapped image with all parameters etc set in place
        """
        if np.shape(rgb_array)[0] < np.shape(rgb_array)[-1]:
            rgb_array = np.moveaxis(rgb_array, 0, -1)
        
        plt.figure(figsize=(6,6))
        plt.imshow(rgb_array)
        plt.axis('off')
        plt.title(f'{sample_id}: AF/{marker_name}')
        plt.savefig(
            os.path.join(md.folders['bg_removed'], f'AF_correction_{sample_id}_{marker_name}.png'), 
            transparent=True
        )
        plt.close()

    
    # the int32 (instead of the classic uint16) is necessary to avoid overflow for values < 0 
    # when subtracting the autofluorescence from the marker channels... took me ages to accept this simple fact,
    # autofluorescence _can_ be stronger than.. autofluorescence + signal?? well, it's real world,
    # things are fucked up. Just clip values and move on.
    clean = signal.astype('int32') - af

    # that was easy peasy, now we export the histogram and the overlapping b
    export_img_signal_histogram(clean, img_id, marker_name)
    combined_clean_AF = combine_clean_AF_channels(
        clean, 
        af, 
        clipping_threshold=clipping_threshold, 
        scale_factor=8
    )

    export_img_signal_AF_overlap(combined_clean_AF, img_id, marker_name)

    np.save(
        os.path.join(md.folders['bg_removed'], f'clean_{marker_name}_{img_id}.npy'),
        (np.clip(clean, 0, np.Inf).astype('uint16') >> 8).astype('uint8')
    )


def single_channel_cleaning(img_id):
    """
    This function assumes that not all channels have been cleaned. Otherwise we are loading the background
    data for nothing...
    """
    #img_id = get_id(img_file)
    img_file = lsd(md.folders['images'], True, f'{img_id}.*tif')
    if len(img_files) != 1:
        return('O SHIT ERROR HERE')
    img_file = img_file[0]
    for i, marker in enumerate(markers):
        print(f'{marker:>10} :', end=' ')
        clean_path = os.path.join(md.folders['bg_removed'], f'clean_{marker}_{img_id}.npy')
        bg = {'cy5': tiff.imread(img_file, key=1), 'tritc': tiff.imread(img_file, key=2)}
        if not os.path.exists(clean_path):
            print('cleaning...')
            signal = tiff.imread(img_file, np.where(np.array(list(md.markers.keys())) == 'ccasp')[0][0])
            subtract_AF(signal, bg[md.marker_channels[marker]])
        else:
            print('exists;')


def extract_remove_bg(channel, square_side):

    sections = np.vstack(swv(channel, (square_side, square_side))[::square_side,::square_side])
    bg = np.min(np.median(sections, axis=(1,2))).astype('uint16')
    channel = np.maximum(channel.astype(np.int32) - bg, 0).astype('uint16') >> 8

    return channel.astype('uint8')



class tissue_sample:
    
    def __init__(self, code):

        self.code = code
        # First we go check what files exist and which don't
        self._md = metadata()
        self._markers = self._md.markers
        self.marker_names = list(self._markers.keys())[3:]
        self.marker_channels = list(self._markers.keys())[:3]

        # this assumes that there is only one file per code (img_id) in the images folder
        self._raw_img_path = lsd(self._md.folders['images'], True, f'{self.code}.*tif')[0]


        self._clean_marker_dict = {
            mk:os.path.exists(os.path.join(self._md.folders['bg_removed'], f'clean_{mk}_{code}.npy')) 
            for mk in self.marker_names
        }

        self._all_files_clean = all(list(self._clean_marker_dict.values()))
        self._segmentation_done = os.path.exists(os.path.join(
            self._md.folders['segmented'], 'cellpose',
            f'whole_segmentation_{code}.npy'
        )) 

        pickle_file_path = os.path.join(md.folders['regionprops'], f'A40_{tile_code}_cell_measures.pickle')
    § 

        self.img = dict()

    def complete_cleaning(self):
        if self._all_files_clean:
            print('background removed for all channels')
        else:
            single_channel_cleaning(self.code)

    def load_dapi(self):
        self.dapi = tiff.imread(self._raw_img_path, key=0)
    
    def create_dapi_mask(self, norm_percentile=95):
        if norm_percentile < 1:
            raise ValueError('norm_percentile is a -percentile- not a quantile.. value entered is <1')
        self.clean_dapi = create_mask(self.dapi, norm_percentile, debug_factor=False)
        #plt.imshow(self.clean_dapi)

    def 

    


In [29]:
mm = tissue_sample('2430')

In [ ]:
md.

In [21]:
md.folders

{'images': '/projects/ag-bozek/lunaphore/data/raw/mouse',
 'bg_removed': '/projects/ag-bozek/lunaphore/data/interim/bg_removed',
 'illumination': '/projects/ag-bozek/lunaphore/data/interim/illumination_correction',
 'segmented': '/projects/ag-bozek/lunaphore/data/interim/segmented',
 'regionprops': '/projects/ag-bozek/lunaphore/data/interim/regionprops'}

In [30]:
mm.__dict__

{'code': '2430',
 '_md': <metadata.metadata at 0x2ae7c4876580>,
 '_markers': {'DAPI': 'DAPI',
  'cy5': 'Cy5 AF',
  'tritc': 'TRITC AF',
  'c-myc': 'c-Myc (Y69)',
  'ecad': 'E-Cadherin 36/E',
  'gfp': 'GFP D5.1',
  'vim': 'Vimentin V9',
  'ccasp': 'Cleaved Caspase3 D175',
  'ki67': 'Ki67 MIB-1',
  'slug': 'SLUG C19G7',
  'ck': 'CK AE1/AE3',
  'fibro': 'Fibronectin E5H6X'},
 'marker_names': ['c-myc',
  'ecad',
  'gfp',
  'vim',
  'ccasp',
  'ki67',
  'slug',
  'ck',
  'fibro'],
 'marker_channels': ['DAPI', 'cy5', 'tritc'],
 '_raw_img_path': '/projects/ag-bozek/lunaphore/data/raw/mouse/20230918_152613_3_9pzeAy_Run 55_A40.2430/20230918_152613_3_9pzeAy_Run 55_A40.2430.tiff',
 '_clean_marker_dict': {'c-myc': True,
  'ecad': True,
  'gfp': True,
  'vim': True,
  'ccasp': True,
  'ki67': True,
  'slug': True,
  'ck': True,
  'fibro': True},
 '_all_files_clean': True,
 '_segmentation_done': True,
 'img': {}}

In [15]:
mm.load_dapi()

In [16]:
mm.create_dapi_mask(norm_percentile=97)

(39640, 39498)
........pool --- 3.4s
(2478, 2469)
........blur --- 0.6s
........norm --- 0.1s
.........bin --- 0.0s
.......blur2 --- 0.1s
........fill --- 0.0s
.....upscale --- 0.2s
.......clean --- 6.4s


In [25]:
lsd(md.folders['segmented'], True)

['/projects/ag-bozek/lunaphore/data/interim/segmented/cellpose/A40_2255_0_dapi1.npy',
 '/projects/ag-bozek/lunaphore/data/interim/segmented/cellpose/A40_2255_1_dapi1.npy',
 '/projects/ag-bozek/lunaphore/data/interim/segmented/cellpose/A40_2255_2_dapi1.npy',
 '/projects/ag-bozek/lunaphore/data/interim/segmented/cellpose/A40_2255_3_dapi1.npy',
 '/projects/ag-bozek/lunaphore/data/interim/segmented/cellpose/A40_2255_4_dapi1.npy',
 '/projects/ag-bozek/lunaphore/data/interim/segmented/cellpose/A40_2255_5_dapi1.npy',
 '/projects/ag-bozek/lunaphore/data/interim/segmented/cellpose/A40_2255_6_dapi1.npy',
 '/projects/ag-bozek/lunaphore/data/interim/segmented/cellpose/A40_2255_7_dapi1.npy',
 '/projects/ag-bozek/lunaphore/data/interim/segmented/cellpose/A40_2255_8_dapi1.npy',
 '/projects/ag-bozek/lunaphore/data/interim/segmented/cellpose/A40_2255_9_dapi1.npy',
 '/projects/ag-bozek/lunaphore/data/interim/segmented/cellpose/A40_2256_0_dapi1.npy',
 '/projects/ag-bozek/lunaphore/data/interim/segmented/

In [ ]:
raw_imgs = lsd(md.folders['images'], True, 'tif')
out_dir = os.path.join('../data/interim/test_dapi_cleaning')
Path(out_dir).mkdir(exist_ok=True, parents=True)

In [ ]:
lsd(md.folders['images'], True, 'tif')

In [ ]:
md.__dict__

In [ ]:
list(md.markers.keys())[:3]

In [ ]:
np.all([os.path.exists(md.folders['bg_removed'], f'clean_{mk}_{code}.npy') for mk in self.marker_names])

In [ ]:
code = '2337'
for mk in list(md.markers.keys())[3:]:
    filepath = os.path.join(md.folders['bg_removed'], f'clean_{mk}_{code}.npy')
    print(os.path.basename(filepath), os.path.exists(filepath))

In [ ]:
[f for f in lsd(md.folders['bg_removed'])]

In [ ]:
raw_imgs

In [ ]:
plt.imshow(~test['bin'])
nc = cv2.connectedComponentsWithStats(1-test['bin'], connectivity=8)[0]
# nc cv2.con

In [ ]:
norm_perc = 95
pool = 10
close_k = 30
open_k = 45
min_size = 2*10**6

for img_file in tqdm(raw_imgs[:1]):
    file_id = get_id(img_file)
    img = tiff.imread(img_file, key=0)
    
    test = create_mask(
        img, 
        norm_percentile=norm_perc, 
        pool_size=pool, 
        size_close_k=close_k, 
        size_open_k=open_k, 
        min_component_size=min_size, 
        debug_factor=4
    )
    
    img[test['fil'] == 0] = 0
    fig, axes = plt.subplots(2, 4, figsize=(20,10))
    ax = axes.ravel()
    for i, k in enumerate(list(test.keys())[:-4]):
        ax[i].imshow(test[k])
        ax[i].set_title(k)
    for a in ax:
        a.axis('off')
    ax[-1].imshow(img)
    ax[-1].set_title('clean')
    plt.tight_layout()
    plt.savefig(os.path.join(out_dir, f'cleaning_{file_id}_n{norm_perc}_p{pool}_c{close_k}_o{open_k}_m{min_size}.png'))
    plt.close()

In [ ]:
class component_coords:
        
    def __init__(self, *args):
        #x_, y_, width, height, area):
        print(len(*args))
        self.args = args
        # if len(*args) == 5:
        #     x_, y_, width, height, area = args
        #     self.x_min = x_
        #     self.y_min = y_
        #     self.width = width
        #     self.height = height
        #     self.x_max = self.x_min + self.width
        #     self.y_max = self.y_min + self.height
        #     self.area = area
    
    def __call__(self):
        return self.args#np.array([self.x_min, self.y_min, self.x_max, self.y_max, self.area])

    def bbox_density(self):
        return self.width * self.height

    
def find_minimum_bounding_box(connected_components_stats, threshold):
    keep = np.array([component_coords(*c)() for c in connected_components_stats[1:] if c[-1] > threshold])
    coords = np.array([keep[:, 0].min(), keep[:, 1].min(), keep[:, 2].max(), keep[:, 3].max()])
    return coords# component_coords(coords[0], coords[1], coords[2] - coords[0], coords[3] - coords[1], coords[-1])

# def rectangle_from_bbox(bounding_box):
#     return patches.Rectangle(
#         coords[]
#     )

In [ ]:
test

In [ ]:
aaa = component_coords(test['components']['s'][0])()

In [ ]:
test['components']['s'][0][0]

In [ ]:
x_, y_, width, height, area = aaa[0]

In [ ]:
chosen_quantile

In [ ]:
plt.figure(figsize=(30,30))
plt.imshow(img[::10,::10])

In [ ]:
plt.figure(figsize=(30,30))
plt.imshow(rraw[::10,::10])

In [ ]:
plt.imshow(remove_small_components(im_upscale.astype('uint8'), 300000)[::10, ::10])

In [ ]:
p = plt.hist(img[img>0].flatten(), bins=2048, log=True)#range(255))

In [ ]:
p = plt.hist(img[img>0].flatten(), bins=range(0,65535,16), log=True)#range(255))

In [ ]:
img.dtype

In [ ]:
test = create_mask(img[::4, ::4], debug_mode=True)
fig, axes = plt.subplots(4,2, figsize=(4,8))
ax = axes.ravel()
for i, k in enumerate(list(test.keys())[:-1]):
    ax[i].imshow(test[k])

In [ ]:
list(test.keys())

In [ ]:
fig, axes = plt.subplots(4,2, figsize=(4,8))
ax = axes.ravel()
for i, k in enumerate(list(test.keys())[:-1]):
    ax[i].imshow(test[k])

In [ ]:
lsd(md.folders['regionprops'])

In [ ]:
news = []
alls = lsd(md.folders['regionprops'])
for s in alls:
    print(os.path.basename(s), '\t', int((time() - os.path.getctime(s)) / 3600))
    if int((time() - os.path.getctime(s)) / 3600) < 10000:
        news.append(s)

#print(news)

In [ ]:
aaa.columns

In [ ]:
import pandas as pd

In [ ]:
aaa = pd.read_pickle(news[0])

In [ ]:
aaa['ecc'] = aaa['axis_minor_length'] / aaa['axis_major_length']

In [ ]:
aaa.loc[:, ['centroid-0', 'centroid-1', 'ecc', 'orientation']]

In [ ]:
np.pi

In [ ]:
2 * aaa.orientation.max()

In [ ]:
p = plt.hist(aaa.orientation*180/np.pi, bins=45)

In [ ]:

fig, ax = plt.subplots(subplot_kw={'projection': 'polar'}, figsize=(10,10))
ax.scatter(aaa.orientation, aaa.ecc, s=0.1, alpha=0.6)

In [ ]:
p = plt.hist(aaa.ecc, bins=50)

In [ ]:

aaa.orientation.min()

In [ ]:
aaa.orientation.max()/np.pi

In [ ]:
np.pi